In [17]:
!rm -rf ~/.cache/pip

In [3]:
!pip install setuptools==70.0.0

  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 72.1.0
    Uninstalling setuptools-72.1.0:
      Successfully uninstalled setuptools-72.1.0


In [1]:
!pip install -r ../requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 21.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 32.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 43.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 27.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import dotenv
assert dotenv.load_dotenv()

In [5]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
#MODEL_ID = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
#MODEL_ID = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0})
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID3, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [5]:
# Instantiate Hugging Face embeddings
hf_embeddings = hf

In [13]:
import os
import pickle
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define the folder containing the PDF files
pdf_folder = "policy_documents"  # Your folder path

# Function to process each PDF and generate embeddings
def process_pdf(pdf_path):
    loader = PyPDFLoader(file_path=pdf_path)
    pdf_docs = loader.load()

    # Add metadata (source) to each document
    for doc in pdf_docs:
        doc.metadata["source"] = pdf_path  # Add file path as the source

    # Split the document into chunks
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300,
        chunk_overlap=50
    )
    splits = text_splitter.split_documents(pdf_docs)

    return splits

# Loop through all PDF files in the folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, pdf_file)

        # Process the PDF
        splits = process_pdf(pdf_path)

        # Create embeddings for the splits
        embeddings = [hf_embeddings.embed_documents([doc.page_content])[0] for doc in splits]

        # Pickle the splits (documents) and embeddings
        pickle_filename = f"{os.path.splitext(pdf_file)[0]}.pkl"
        with open(pickle_filename, "wb") as f:
            pickle.dump({"documents": splits, "embeddings": embeddings}, f)

        print(f"Pickled data for {pdf_file} to {pickle_filename}")

Pickled data for Spain.pdf to Spain.pkl
Pickled data for South-Africa.pdf to South-Africa.pkl
Pickled data for Poland.pdf to Poland.pkl
Pickled data for Latvia.pdf to Latvia.pkl
Pickled data for Ireland.pdf to Ireland.pkl
Pickled data for Oman.pdf to Oman.pkl
Pickled data for Qatar.pdf to Qatar.pkl
Pickled data for New-Zealand.pdf to New-Zealand.pkl
Pickled data for Australia.pdf to Australia.pkl
Pickled data for Serbia.pdf to Serbia.pkl
Pickled data for North-Macedonia.pdf to North-Macedonia.pkl
Pickled data for Belgium-Flemish.pdf to Belgium-Flemish.pkl
Pickled data for Finland.pdf to Finland.pkl
Pickled data for Georgia.pdf to Georgia.pkl
Pickled data for Slovenia.pdf to Slovenia.pkl
Pickled data for Hong-Kong-SAR.pdf to Hong-Kong-SAR.pkl
Pickled data for Austria.pdf to Austria.pkl
Pickled data for Italy.pdf to Italy.pkl
Pickled data for Northern-Ireland.pdf to Northern-Ireland.pkl
Pickled data for Croatia.pdf to Croatia.pkl
Pickled data for Uzbekistan.pdf to Uzbekistan.pkl
Pickled 

In [26]:
from langchain_community.vectorstores import Chroma

country_name = "Singapore"
# Load the pickled documents
with open(f"embeddings/{country_name}.pkl", "rb") as f:
    data = pickle.load(f)

documents = data["documents"]

# Recreate the vectorstore from the loaded documents (embedding will be computed by `hf_embeddings`)
vectorstore = Chroma.from_documents(documents=documents, embedding=hf_embeddings)

# Define a directory to persist the vectorstore
persist_directory = f"./chroma_store/{country_name}"

# Create the vectorstore and persist it
vectorstore = Chroma.from_documents(documents=documents, embedding=hf_embeddings, persist_directory=persist_directory)

# Persist the vectorstore
vectorstore.persist()

# Now you can use the vectorstore as usual
# retriever = vectorstore.as_retriever()
# question = f"What is the (reading) curriculum for {country_name}?"
# results = retriever.get_relevant_documents(query)
# retriever = vectorstore.as_retriever()


/tmp/ipykernel_27797/4246369355.py:20: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [24]:
import requests
from io import BytesIO
from langchain_core.tools import tool
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock
from langchain.load import dumps, loads
from pydantic import BaseModel
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate three 
different versions of the given user question to retrieve relevant documents from a vector 
database. Provide these alternative questions separated by newlines. 
Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={'temperature': 0, "max_tokens": 8192})
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

# Retrieve
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})

# RAG Chain
template = """Answer the following question based on this context:

{context}

Question: {question}

Please always provide direct citations in your summary, but don't mention the source.

Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}
    | prompt
    | ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", model_kwargs={'temperature': 0, "max_tokens": 8192})
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'The reading curriculum in Singapore is comprehensive and integrated into the broader English language curriculum. Key aspects include:\n\n1. A focus on literacy development as a core part of English instruction, aiming to help students "use English effectively and appropriately" by the end of primary school.\n\n2. The 10-year English Language Syllabus 2010, which covers learning outcomes from primary to secondary levels, aiming to develop critical listening, reading, and viewing skills.\n\n3. The Strategies for English Language Learning and Reading (STELLAR) program in primary schools, which aims to "develop in students a strong foundation in the English language and a love for reading."\n\n4. Use of various reading strategies including "decoding through phonics, shared reading, close reading and viewing, critical reading, viewing and appreciation, and extensive reading and viewing."\n\n5. Emphasis on reading for enjoyment and encouraging students to "read widely and learn to analyze 

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# URL of the PDF document
pdf_url = "https://pirls2021.org/wp-content/uploads/2023/05/P21_MP_Ch3-sample-design.pdf"  # Replace with actual URL

# Load the PDF from the URL
loader = PyPDFLoader(file_path=pdf_url)
pdf_docs = loader.load()

# Add metadata (source) to each document
for doc in pdf_docs:
    doc.metadata["source"] = pdf_url  # Add URL as the source

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(pdf_docs)

# Create the vectorstore using Hugging Face embeddings and IDs
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

In [7]:
import nltk
print(nltk.__version__)

3.9.1


In [8]:
import nltk

nltk.download('punkt')
nltk.download('punkt', download_dir='/home/ec2-user/nltk_data')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [9]:
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Path to the Excel file
excel_path = "https://pirls2021.org/wp-content/uploads/2022/files/1_1-2_achievement-results-1.xlsx"  # Replace with your actual file path

# Load the Excel file
loader = UnstructuredExcelLoader(file_path=excel_path, mode="elements")
excel_docs = loader.load()

# Function to clean metadata
def clean_metadata(metadata):
    for key, value in metadata.items():
        if isinstance(value, list):
            metadata[key] = ', '.join(value)  # Convert list to comma-separated string
    return metadata

# Clean metadata for each document
for doc in excel_docs:
    doc.metadata = clean_metadata(doc.metadata)

# Split the document into chunks if needed
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(excel_docs)

# Create the vectorstore using Hugging Face embeddings and IDs
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

In [11]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader


target_url = "https://pirls2021.org/results/international-benchmarks/percentages"
loader = WebBaseLoader(
    web_paths=(target_url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer("p")
    ),
)
blog_docs = loader.load()

# Add metadata for web source (URL)
for doc in blog_docs:
    doc.metadata["source"] = target_url

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)



# Make splits
splits = text_splitter.split_documents(blog_docs)

# Create the vectorstore using Hugging Face embeddings and IDs
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate three 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [18]:
from pydantic import BaseModel

class Config:
    arbitrary_types_allowed = True

In [20]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "Who are reading scores weighted in PIRLS 2021?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

14

In [23]:
from operator import itemgetter
# from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}

The answer to the user's question in markdown format. 
Each paragraph starts with an emoji and a heading in capital letters. 
Use bullet points (starting with emoji) and tables to increase readability. 
Cite explicitly and cite your sources with a link. The hyperlink should be directly added to the quote and a footnote added at the end. 
Also refer to sources (links) that are mentioned in relation to your citations.
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'📊 SAMPLING WEIGHTS IN PIRLS 2021\n\n🔢 OVERVIEW OF WEIGHTING PROCESS\n- Student reading scores in PIRLS 2021 are weighted using a complex sampling design to ensure representative results\n- Weights account for selection probabilities and non-response at multiple levels:\n  • School level\n  • Class level \n  • Student level\n\n🏫 SCHOOL WEIGHTING\n- Schools are sampled with probability proportional to size\n- Basic school weight is the inverse of selection probability\n- Adjusted for non-participating schools\n\n📚 CLASS WEIGHTING  \n- Classes within sampled schools are randomly selected\n- Class weights account for number of classes selected per school\n- Adjusted for non-participating classes\n\n👨\u200d🎓 STUDENT WEIGHTING\n- All students in selected classes are included\n- Student weights account for student non-response within classes\n\n🧮 FINAL WEIGHT CALCULATION\n- Overall student weight is the product of school, class, and student weight components\n- As stated in the source: "The 